In [ ]:
%cd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab Notebooks/WSSS
%ls

/root
Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/WSSS
AdelaiDet/                   res38_cls.pth
ADL_notok.ipynb              SEAM/
AuxSegNet/                   SESS/
BANA/                        src/
BBAM/                        test_img2.jpg
BBAM_notok.ipynb             test_img.jpg
BESTIE/                      TokenCut/
BoxInst_notok.ipynb          TokenCut_ok.ipynb
CCAM/                        transcam/
CCAM_ok.ipynb                transcam_6485.pth
COCO2YOLO/                   transcam_deeplab.ipynb
Conformer_small_patch16.pth  transcam_instance_ok.ipynb
data_voc/                    transcam_instance_v2_ok.ipynb
DiscoBox/                    transcam_instance_v3_ok.ipynb
DiscoBox_notok.ipynb         transcam_masks_ok.ipynb
externals/                   transcam_ok.ipynb
instances_val2017.json       transcam_train.ipynb
instance_train_ok.ipynb      transcam_wheat_ok.ipynb
isim/                        transcam_wheat_train.ipynb
isim_notok.ipynb             wheat

In [ ]:
%cd transcam
!pip install -r requirements.txt
%ls

/content/drive/MyDrive/Colab Notebooks/WSSS/transcam
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-9o_y862k
  Running command git clone --filter=blob:none --quiet https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-9o_y862k
  Resolved https://github.com/lucasb-eyer/pydensecrf.git to commit 0d53acbcf5123d4c88040fe68fbb9805fc5b2fb9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 881.2/881.9 MB 188.4 MB/s eta 0:00:01tcmalloc: large alloc 1102397440 bytes == 0x38756000 @  0x7f2959e43615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d

In [ ]:
import numpy as np
import torch
import os

from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
import voc12.data
from tool import pyutils, imutils
import argparse
import importlib
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

from torch.utils.data import Dataset
import PIL.Image
import os.path
import scipy.misc

from glob import glob

In [ ]:
# Patch to images
train_folder = '../wheat/images/train/'
print('Number of files in the train folder', len(os.listdir(train_folder)))

Number of files in the train folder 4067


In [ ]:
# TransCAM parameters
num_workers = 0;
lr = 5e-5;
wt_dec = 5e-4;
batch_size = 2;
max_epoches = 20;
network = "network.conformer_CAM";
arch = "sm";

session_name = "TransCAM_mytest_WHEAT_v13"
crop_size = 512;
weights = '../Conformer_small_patch16.pth'
tblog_dir ='./tblog'
save_dir ='./';


In [ ]:
""" Functions """
# ['hotdog, food, furniture, people, pets]
def load_image_label_list_from_npy(img_name_list):
  cls_labels_dict = {};

  for i in img_name_list:
    if ('IMG' in i): 
      cls_labels_dict[i] = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32);
    else:
      cls_labels_dict[i] = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32);

  return [cls_labels_dict[img_name] for img_name in img_name_list]

IMG_FOLDER_NAME = train_folder;
def get_img_path(img_name):
    return os.path.join(IMG_FOLDER_NAME, img_name + '.jpg')

def load_img_name_list(dataset_path):
  img_gt_name_list = sorted(glob(os.path.join(dataset_path, "*.jpg")));
  img_name_list = [img_gt_name.split('/')[-1].split('.')[0] for img_gt_name in img_gt_name_list];
  return img_name_list


""" Classes """
class VOC12ImageDataset(Dataset):

    def __init__(self, img_name_list_path, transform=None):
        self.img_name_list = load_img_name_list(img_name_list_path) # Image names without .jpg
        self.transform = transform

    def __len__(self):
        return len(self.img_name_list)

    def __getitem__(self, idx):
        name = self.img_name_list[idx]

        img = PIL.Image.open(get_img_path(name)).convert("RGB") # Open image

        if self.transform:
            img = self.transform(img)

        return name, img
        
class VOC12ClsDataset(VOC12ImageDataset):

    def __init__(self, img_name_list_path, transform=None):
        super().__init__(img_name_list_path, transform)
        self.label_list = load_image_label_list_from_npy(self.img_name_list) # Massive of the class vectors for each image
 
    def __getitem__(self, idx):
        name, img = super().__getitem__(idx)

        label = torch.from_numpy(self.label_list[idx])

        return name, img, label # Image name; Image; Class vector



In [ ]:

pyutils.Logger(session_name + '.log')

model = getattr(importlib.import_module(network), 'Net_' + arch)()

tblogger = SummaryWriter(tblog_dir)

train_dataset = VOC12ClsDataset(train_folder,
                                            transform=transforms.Compose([
                                                imutils.RandomResizeLong(320, 640),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3,
                                                                      hue=0.1),
                                                np.asarray,
                                                imutils.Normalize(),
                                                imutils.RandomCrop(crop_size),
                                                imutils.HWC_to_CHW,
                                                torch.from_numpy
                                            ]))

train_data_loader = DataLoader(train_dataset, batch_size=batch_size,
                                shuffle=True, num_workers=0, pin_memory=True, drop_last=True)

optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wt_dec, eps=1e-8)

checkpoint = torch.load(weights, map_location='cpu')
if 'model' in checkpoint.keys():
    checkpoint = checkpoint['model']
else:
    checkpoint = checkpoint
model_dict = model.state_dict()
for k in ['trans_cls_head.weight', 'trans_cls_head.bias']:
    print(f"Removing key {k} from pretrained checkpoint")
    del checkpoint[k]
for k in ['conv_cls_head.weight', 'conv_cls_head.bias']:
    print(f"Removing key {k} from pretrained checkpoint")
    del checkpoint[k]
pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

model = torch.nn.DataParallel(model).cuda()
model.train()

avg_meter = pyutils.AverageMeter('loss')

TRAIN_LOSS = [];
for ep in range(max_epoches):
    print('ep num', ep)
    for iter, pack in enumerate(train_data_loader):
        _, img, label = pack
        N, C, H, W = img.size()
        bg_score = torch.ones((N, 1))
        label = torch.cat((bg_score, label), dim=1)
        label = label.cuda(non_blocking=True).unsqueeze(2).unsqueeze(3)

        logits_conv, logits_trans, cams = model('transcam', img)

        loss = F.multilabel_soft_margin_loss((logits_conv + logits_trans).unsqueeze(2).unsqueeze(3)[:, 1:, :, :], label[:, 1:, :, :])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ##########################
        # print('epoch: %5d' % ep,
        #       'iter: %5d' % iter,
        #       'loss: %.4f' % loss, flush=True)
        TRAIN_LOSS.append(loss);
        ##########################
        avg_meter.add({'loss': loss.item()})
    else:
        print('epoch: %5d' % ep,
              'loss: %.4f' % avg_meter.get('loss'), flush=True)
        avg_meter.pop()
    torch.save(model.module.state_dict(), os.path.join(save_dir, session_name + '.pth'))

torch.save(model.module.state_dict(), os.path.join(save_dir, session_name + '.pth'))

Removing key trans_cls_head.weight from pretrained checkpoint
Removing key trans_cls_head.bias from pretrained checkpoint
Removing key conv_cls_head.weight from pretrained checkpoint
Removing key conv_cls_head.bias from pretrained checkpoint
ep num 0
epoch:     0 loss: 0.0148
ep num 1
epoch:     1 loss: 0.0024
ep num 2
epoch:     2 loss: 0.0000
ep num 3
epoch:     3 loss: 0.0027
ep num 4
epoch:     4 loss: 0.0025
ep num 5
epoch:     5 loss: 0.0018
ep num 6
epoch:     6 loss: 0.0007
ep num 7
epoch:     7 loss: 0.0027
ep num 8
epoch:     8 loss: 0.0011
ep num 9
